# Task IV - Adding Social Influence to the Market Model

This notebook implements a customer network using NetworkX and analyzes how social influence affects seller pricing and profitability.

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

## 1. Load Data from Previous Tasks

In [ ]:
seller_config_df = pd.read_csv('seller_config.csv')
initial_profit_df = pd.read_csv('initial_profit_summary.csv')
nash_equilibrium_df = pd.read_csv('nash_equilibrium_results.csv')

print("Seller Configuration (Task II):")
print(seller_config_df.to_string(index=False))
print("\n\nNash Equilibrium Results (Task III):")
print(nash_equilibrium_df.to_string(index=False))

## 2. Build Customer Network with Influencers

In [ ]:
np.random.seed(42)

cleaned_data = pd.read_csv('../task-I/cleaned_data.csv')
customers = cleaned_data['Customer ID'].dropna().unique()
num_customers = min(len(customers), 500)
customers = np.random.choice(customers, size=num_customers, replace=False)

print(f"Creating network with {num_customers} customers...")

G = nx.Graph()
G.add_nodes_from(customers)

for customer in customers:
    num_connections = np.random.randint(1, 8)
    potential_neighbors = np.random.choice(customers, size=num_connections, replace=False)
    for neighbor in potential_neighbors:
        if customer != neighbor:
            G.add_edge(customer, neighbor, weight=np.random.uniform(0.5, 1.0))

print(f"\nNetwork Statistics:")
print(f"  Number of nodes: {G.number_of_nodes()}")
print(f"  Number of edges: {G.number_of_edges()}")
print(f"  Network density: {nx.density(G):.4f}")
print(f"  Average clustering coefficient: {nx.average_clustering(G):.4f}")

if nx.is_connected(G):
    print(f"  Average shortest path length: {nx.average_shortest_path_length(G):.4f}")
else:
    print(f"  Network has {nx.number_connected_components(G)} connected components")

## 3. Identify Influencers Using Centrality Measures

In [ ]:
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

combined_centrality = {}
for node in G.nodes():
    combined_centrality[node] = (
        0.3 * degree_centrality[node] +
        0.3 * betweenness_centrality[node] +
        0.2 * closeness_centrality[node] +
        0.2 * eigenvector_centrality[node]
    )

num_influencers = max(10, int(num_customers * 0.1))
influencers = sorted(combined_centrality.items(), key=lambda x: x[1], reverse=True)[:num_influencers]
influencer_ids = set([inf[0] for inf in influencers])

print(f"\nIdentified {num_influencers} Influencers (Top Centrality Scores):")
print(f"{'Rank':<6} {'Customer ID':<15} {'Combined Score':<20}")
print("=" * 40)
for rank, (cust_id, score) in enumerate(influencers[:10], 1):
    print(f"{rank:<6} {int(cust_id):<15} {score:<20.6f}")

influencer_influence = {}
for customer in G.nodes():
    if customer in influencer_ids:
        influencer_influence[customer] = 2.0
    else:
        influencer_influence[customer] = 1.0

## 4. Calculate Network Influence Propagation

In [ ]:
def calculate_network_influence(graph, influencer_influence, depth=2):
    node_influence = {node: 0 for node in graph.nodes()}
    
    for node in graph.nodes():
        if influencer_influence[node] > 1.0:
            node_influence[node] += influencer_influence[node]
    
    for current_depth in range(1, depth + 1):
        influence_to_propagate = {node: 0 for node in graph.nodes()}
        
        for node in graph.nodes():
            if node_influence[node] > 0:
                neighbors = list(graph.neighbors(node))
                for neighbor in neighbors:
                    decay_factor = 1.0 / (1.0 + current_depth * 0.5)
                    influence_to_propagate[neighbor] += node_influence[node] * decay_factor / len(neighbors)
        
        for node in graph.nodes():
            node_influence[node] += influence_to_propagate[node]
    
    normalization_factor = max(node_influence.values()) if node_influence.values() else 1.0
    node_influence = {node: (influence / normalization_factor * 100) for node, influence in node_influence.items()}
    
    return node_influence

network_influence = calculate_network_influence(G, influencer_influence, depth=2)

influence_scores = list(network_influence.values())
print(f"\nNetwork Influence Score Statistics:")
print(f"  Mean: {np.mean(influence_scores):.2f}")
print(f"  Std: {np.std(influence_scores):.2f}")
print(f"  Min: {np.min(influence_scores):.2f}")
print(f"  Max: {np.max(influence_scores):.2f}")

print(f"\nTop 10 Customers by Network Influence:")
top_influenced = sorted(network_influence.items(), key=lambda x: x[1], reverse=True)[:10]
for rank, (cust_id, inf_score) in enumerate(top_influenced, 1):
    influencer_flag = "(INFLUENCER)" if cust_id in influencer_ids else ""
    print(f"  {rank}. Customer {int(cust_id)}: {inf_score:.2f} {influencer_flag}")

## 5. Updated Demand Function with Network Influence

In [ ]:
class InfluenceAwareSeller:
    def __init__(self, name, price, cost, ad_budget, base_demand, items, initial_profit):
        self.name = name
        self.price = price
        self.cost = cost
        self.ad_budget = ad_budget
        self.base_demand = base_demand
        self.items = items
        self.initial_profit = initial_profit
        self.profit = 0
        self.demand = 0
    
    def calculate_demand(self, competitor_prices, alpha, beta, gamma, network_influence_level):
        ad_effect = alpha * self.ad_budget
        
        price_effects = []
        for comp_price in competitor_prices:
            price_effect = beta * (self.price - comp_price)
            price_effects.append(price_effect)
        avg_price_effect = np.mean(price_effects) if price_effects else 0
        
        influence_effect = gamma * 100 * network_influence_level
        
        demand = self.base_demand + ad_effect + avg_price_effect + influence_effect
        return max(0, demand)

sellers_config = [
    ('Seller_A', 14.80, 2.0, 50, 84277, 30, -123574.74),
    ('Seller_B', 14.70, 2.5, 50, 25083, 61, 1811.04),
    ('Seller_C', 14.90, 3.0, 50, 11465, 35, 84119.69)
]
sellers = [InfluenceAwareSeller(*config) for config in sellers_config]

print("Influence-Aware Sellers Initialized")
for seller in sellers:
    print(f"  {seller.name}: p=${seller.price:.2f}, cost=${seller.cost:.2f}, ad_budget=${seller.ad_budget:.2f}")

## 6. Analyze Impact of Varying Social Influence

In [ ]:
alpha = 0.05
beta = 15.0
gamma = 0.8

influence_multipliers = np.linspace(0.1, 2.0, 10)
results = []

print("\n" + "="*80)
print("IMPACT OF SOCIAL INFLUENCE ON SELLER PERFORMANCE")
print("="*80)

for multiplier in influence_multipliers:
    print(f"\nNetwork Influence Level: {multiplier:.2f}x (1.0 = baseline)")
    print("-" * 80)
    
    seller_data = {}
    
    for seller in sellers:
        competitor_prices = [s.price for s in sellers if s.name != seller.name]
        
        demand = seller.calculate_demand(
            competitor_prices, alpha, beta, gamma, 
            network_influence_level=multiplier
        )
        
        revenue = (seller.price - seller.cost) * demand
        profit = revenue - seller.ad_budget
        
        seller_data[seller.name] = {
            'demand': demand,
            'revenue': revenue,
            'profit': profit,
            'margin_per_unit': seller.price - seller.cost
        }
        
        print(f"{seller.name}:")
        print(f"  Demand: {demand:,.2f} units")
        print(f"  Revenue (before ad): ${revenue:,.2f}")
        print(f"  Profit (after ad): ${profit:,.2f}")
    
    for seller_name, data in seller_data.items():
        results.append({
            'Influence_Multiplier': multiplier,
            'Seller': seller_name,
            'Demand': data['demand'],
            'Profit': data['profit'],
            'Revenue': data['revenue']
        })

results_df = pd.DataFrame(results)
print("\n" + "="*80)

## 7. Comparative Analysis: With vs Without Social Influence

In [ ]:
print("\n" + "="*80)
print("COMPARATIVE ANALYSIS: Impact of Social Influence")
print("="*80)

no_influence_results = results_df[results_df['Influence_Multiplier'] == 0.1].copy()
high_influence_results = results_df[results_df['Influence_Multiplier'] == 2.0].copy()

comparison = pd.DataFrame()
comparison['Seller'] = no_influence_results['Seller'].values
comparison['Demand_Low_Influence'] = no_influence_results['Demand'].values
comparison['Demand_High_Influence'] = high_influence_results['Demand'].values
comparison['Demand_Change_%'] = ((high_influence_results['Demand'].values - no_influence_results['Demand'].values) / no_influence_results['Demand'].values * 100)
comparison['Profit_Low_Influence'] = no_influence_results['Profit'].values
comparison['Profit_High_Influence'] = high_influence_results['Profit'].values
comparison['Profit_Change_%'] = ((high_influence_results['Profit'].values - no_influence_results['Profit'].values) / no_influence_results['Profit'].values * 100)

print("\n" + comparison.to_string(index=False))

print("\n\nKey Insights:")
print("-" * 80)
for idx, row in comparison.iterrows():
    seller = row['Seller']
    demand_change = row['Demand_Change_%']
    profit_change = row['Profit_Change_%']
    print(f"\n{seller}:")
    print(f"  Demand increases by {demand_change:.2f}% with higher social influence")
    print(f"  Profit changes by {profit_change:.2f}% with higher social influence")
    if profit_change > 0:
        print(f"  → Social influence BENEFITS this seller")
    else:
        print(f"  → Social influence HARMS this seller")

## 8. Save Results and Analysis

In [ ]:
results_df.to_csv('social_influence_analysis.csv', index=False)
comparison.to_csv('influence_comparison.csv', index=False)

print("\nNetwork Statistics Summary:")
network_stats = {
    'Metric': [
        'Total Customers',
        'Total Network Edges',
        'Network Density',
        'Average Clustering Coefficient',
        'Number of Influencers',
        'Influencer Percentage',
        'Average Network Influence Score',
        'Max Network Influence Score'
    ],
    'Value': [
        G.number_of_nodes(),
        G.number_of_edges(),
        f"{nx.density(G):.4f}",
        f"{nx.average_clustering(G):.4f}",
        num_influencers,
        f"{(num_influencers / num_customers * 100):.2f}%",
        f"{np.mean(influence_scores):.2f}",
        f"{np.max(influence_scores):.2f}"
    ]
}
network_stats_df = pd.DataFrame(network_stats)
network_stats_df.to_csv('network_statistics.csv', index=False)

print("\n" + network_stats_df.to_string(index=False))

print(f"\n\nFiles saved:")
print(f"  - social_influence_analysis.csv")
print(f"  - influence_comparison.csv")
print(f"  - network_statistics.csv")

## 9. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("TASK IV SUMMARY: SOCIAL INFLUENCE IN E-COMMERCE MARKETS")
print("="*80)

print("\n1. NETWORK STRUCTURE:")
print(f"   - Built a customer network with {G.number_of_nodes()} customers")
print(f"   - Network has {G.number_of_edges()} connection edges")
print(f"   - Identified {num_influencers} key influencers ({num_influencers/num_customers*100:.1f}% of network)")
print(f"   - Network density: {nx.density(G):.4f} (shows moderate connectivity)")

print("\n2. INFLUENCE PROPAGATION:")
print(f"   - Influencers have 2.0x base influence multiplier")
print(f"   - Influence decays with network distance (depth=2)")
print(f"   - Average network influence score: {np.mean(influence_scores):.2f}")
print(f"   - Network influence ranges from {np.min(influence_scores):.2f} to {np.max(influence_scores):.2f}")

print("\n3. MARKET IMPACT ANALYSIS:")
avg_demand_change = comparison['Demand_Change_%'].mean()
avg_profit_change = comparison['Profit_Change_%'].mean()
print(f"   - Average demand increase: {avg_demand_change:.2f}%")
print(f"   - Average profit change: {avg_profit_change:.2f}%")

print("\n4. SELLER-SPECIFIC INSIGHTS:")
for idx, row in comparison.iterrows():
    print(f"   - {row['Seller']}: Demand +{row['Demand_Change_%']:.1f}%, Profit {row['Profit_Change_%']:+.1f}%")

print("\n5. STRATEGIC IMPLICATIONS:")
print("   - Social influence creates positive externality for all sellers")
print("   - Sellers benefit from network effects and word-of-mouth")
print("   - Higher influence multipliers expand total market demand")
print("   - Influencer targeting becomes critical for market growth")

print("\n" + "="*80)